In [1]:
import re
import numpy as np
import math
from matplotlib import pyplot as plt
import copy
from functools import lru_cache

In [4]:
with open("example.txt") as f:
    data = f.read()
    lines = data.strip().split("\n")
    records = data.strip().split("\n\n")

In [29]:
with open("input.txt") as f:
    data = f.read()
    lines = data.strip().split("\n")   
    records = data.strip().split("\n\n")

In [30]:
#board = np.array([list(x) for x in lines])
#plt.matshow(screen)

In [31]:
valves = {}
paths = {}
start = "AA"
for line in lines:
    name = line.split()[1]
    if start is None:
        start = name
    rate = int(re.findall(r"\d+",line)[0])
    other_connections = [x[0:2] for x in line.split()[9:]]
    print(other_connections)
    valves[name] = rate
    paths[name] = other_connections

valve_to_num = {}
num_to_valve = {}
i = 0
for k in sorted(valves.keys()):
    if valves[k] == 0:
        continue
    valve_to_num[k] = 1<<i
    num_to_valve[valve_to_num[k]] = k
    i+=1

@lru_cache
def num_to_rate(num):
    global valves
    rate = 0
    for i in range(len(valve_to_num)):
        if 1<<i & num:
            rate+=valves[num_to_valve[1<<i]]
    return rate

['MU', 'PH']
['KB', 'MH']
['GC', 'FI']
['UN', 'CN']
['AA', 'IF']
['IF', 'WB']
['BL', 'DJ']
['BO', 'RU']
['AZ', 'IF']
['TD', 'AZ']
['RF', 'GU']
['MH', 'MU']
['DJ', 'UN']
['VJ', 'BA', 'YP']
['QF', 'PI', 'AS', 'PE']
['VJ', 'UM', 'CN']
['GA', 'CQ']
['IE', 'WQ']
['IE', 'IF']
['CQ', 'MH']
['YP', 'UM', 'XI', 'XA']
['KB', 'BL', 'GA']
['DR', 'BT']
['IF', 'FI']
['RQ', 'NT']
['FI', 'AZ']
['TD', 'JL']
['ME', 'TD']
['AB']
['AR', 'JL', 'HK', 'PE', 'XI']
['ME', 'AB']
['WQ', 'XD']
['AA', 'FS']
['AA', 'AZ']
['HH', 'MH']
['TD', 'AA']
['HH', 'CO', 'AS', 'HK', 'PQ']
['OQ', 'BO', 'PH']
['UN', 'BA']
['FS', 'NW', 'SS', 'SY']
['IE', 'GU']
['PI', 'MR', 'AR', 'CO', 'DI']
['CQ', 'MU']
['AA', 'FI']
['QF', 'GC', 'OQ', 'WB', 'PQ']
['SY', 'UU', 'NA']
['BT', 'RQ']
['ME', 'NA']
['IE', 'DI']
['TD', 'MR']
['BT', 'NW']
['XA', 'FI']
['UU', 'DR', 'SS']
['WQ', 'GU']
['NT', 'XD', 'RF']


In [32]:
start

'AA'

In [33]:
print(valve_to_num)
print(valves)

{'AZ': 1, 'BT': 2, 'CQ': 4, 'FI': 8, 'GU': 16, 'IE': 32, 'IF': 64, 'ME': 128, 'MH': 256, 'MU': 512, 'RQ': 1024, 'RU': 2048, 'TD': 4096, 'UN': 8192, 'WQ': 16384}
{'NA': 0, 'NW': 0, 'MR': 0, 'XD': 0, 'HK': 0, 'JL': 0, 'RQ': 13, 'AB': 0, 'PE': 0, 'QF': 0, 'BA': 0, 'SY': 0, 'NT': 0, 'GU': 21, 'AZ': 12, 'WQ': 23, 'DR': 0, 'UM': 0, 'XI': 0, 'SS': 0, 'IE': 22, 'BT': 24, 'GA': 0, 'AR': 0, 'DJ': 0, 'PI': 0, 'WB': 0, 'OQ': 0, 'RU': 19, 'IF': 7, 'BO': 0, 'CN': 0, 'HH': 0, 'AS': 0, 'FS': 0, 'PQ': 0, 'AA': 0, 'ME': 18, 'RF': 0, 'MH': 8, 'YP': 0, 'FI': 11, 'UU': 0, 'CO': 0, 'TD': 16, 'MU': 15, 'BL': 0, 'PH': 0, 'XA': 0, 'GC': 0, 'KB': 0, 'DI': 0, 'CQ': 9, 'VJ': 0, 'UN': 20}


In [34]:
# part 1
states = [(start,0,0)]
#state = (loc,preasure,flipped)
bests = {}
MAX_TIME = 26
for i in range(1,MAX_TIME+1):
    new_state = []
    for state in states:
        loc,preasure,flipped = state

        if (loc,flipped) in bests and preasure <= bests[(loc,flipped)]:
            continue
        
        bests[(loc,flipped)] = preasure
        
        if loc in valve_to_num and valve_to_num[loc] & flipped == 0:
            new_flipped = flipped | valve_to_num[loc]
            new_state.append((loc,preasure+valves[loc]*(MAX_TIME-i),new_flipped))
        
        for dest in paths[loc]:
            new_state.append((dest,preasure,flipped))
        
    states = new_state
    print(i,len(states))           

1 5
2 10
3 25
4 46
5 59
6 117
7 203
8 237
9 390
10 624
11 842
12 1136
13 1642
14 2334
15 3304
16 4544
17 6203
18 8599
19 11736
20 15868
21 21584
22 28901
23 38290
24 51288
25 68574
26 90873


In [35]:
best_flipped = {}
for (loc,flipped),value in bests.items():
    if flipped not in best_flipped:
        best_flipped[flipped] = value
    elif value > best_flipped[flipped]:
        best_flipped[flipped] = value

In [38]:
print(len(best_flipped))

4808


In [36]:
best = (0,0,0)
for k1 in best_flipped:
    for k2 in best_flipped:
        if k1 & k2 == 0:
            if best_flipped[k1] + best_flipped[k2] > best[0]:
                best = (best_flipped[k1] + best_flipped[k2],k1,k2)

In [40]:
print(best)
print(hex(best[1]+best[2]))

(2911, 25712, 4999)
0x77f7


In [11]:
print(max((x[1] for x in states)))

1651


In [12]:
"""
1 3
2 9
3 25
4 71
5 176
6 445
7 1036
8 2487
9 5684
10 13566
11 30583
12 72542
13 163469
387979
874657
2075430
4678789
11102556
25031007

1 3
2 9
3 25
4 71
5 198
6 556
7 1533
8 4247
9 11566
10 31616
11 85205
12 230122
13 614709
14 1642866
15 4354546
16 11532568"""

'\n1 3\n2 9\n3 25\n4 71\n5 176\n6 445\n7 1036\n8 2487\n9 5684\n10 13566\n11 30583\n12 72542\n13 163469\n387979\n874657\n2075430\n4678789\n11102556\n25031007\n\n1 3\n2 9\n3 25\n4 71\n5 198\n6 556\n7 1533\n8 4247\n9 11566\n10 31616\n11 85205\n12 230122\n13 614709\n14 1642866\n15 4354546\n16 11532568'

In [15]:
#def visit(target,valves,visited):
    
non_zero_valves = {k:v for k,v in valves.items() if v > 0}

        

def distanceTo (target,valves):
    visited = {target:0}
    cur = target
    to_visit = set([target])
    while len(visited) < len(valves) and len(to_visit) > 0:
        update = []
        for new in to_visit:
            for k in valves[new]:
                if k not in visited:
                    update.append(k)
                    visited[k] = visited[new]+1
        to_visit = update
    if len(to_visit) == 0:
        print("to_visit is empty")
    del visited[target]
    return visited

shorter_caves = {}
for cave in non_zero_valves:
    shorter_caves[cave] = distanceTo(cave,paths)
shorter_caves [start] = distanceTo(start,paths)

In [13]:
# part 2
print(start)
states = [(start,start,0,0)]
#state = (loc,preasure,flipped)
bests = {}
MAX_TIME = 26
not_printed_yet = False
states = []*(MAX_TIME+1)
for i in range(1,MAX_TIME+1):
    for state in states[i]:
        loc,loc1,preasure,flipped = state

        if (loc,loc1,flipped) in bests and preasure <= bests[(loc,loc1,flipped)]:
            continue
        
        bests[(loc,loc1,flipped)] = copy.copy(preasure)
        if not_printed_yet and flipped > 40:
            not_printed_yet = True
            print("flipped finished in round",i)
        if loc in valve_to_num and valve_to_num[loc] & flipped == 0:
            new_flipped = flipped | valve_to_num[loc]
            new_state.append((loc,loc1,preasure+valves[loc]*(MAX_TIME-i),new_flipped))

            if loc1 in valve_to_num and valve_to_num[loc1] & new_flipped == 0:
                new_flipped1 = new_flipped | valve_to_num[loc1]
                new_state.append((loc,loc1,preasure+(valves[loc]*(MAX_TIME-i))+(valves[loc1]*(MAX_TIME-i)),new_flipped1))

            for dest1 in paths[loc1]:
                new_state.append((loc,dest1,preasure+valves[loc]*(MAX_TIME-i),new_flipped))
        
        for dest in paths[loc]:
            #new_state.append((dest,loc1,preasure,flipped))
            if loc1 in valve_to_num and valve_to_num[loc1] & flipped == 0:
                new_flipped1 = flipped | valve_to_num[loc1]
                new_state.append((dest,loc1,preasure+(valves[loc1]*(MAX_TIME-i)),new_flipped1))

            for dest1 in paths[loc1]:
                new_state.append((dest,dest1,preasure,flipped))
  
        
    #print(states)
    print(i,len(states))   
    print(max((x[2] for x in states)))
    if len(new_state) > 0:
        states = new_state

AA


IndexError: list index out of range

In [150]:
print(max((x[2] for x in states)))

1253


In [166]:
sum(valves.values())*26

6188

In [154]:
print(max(bests.values()))

1707


In [ ]:
# part 2, guess 2973
# part 2, guess 2793 (too low)

In [ ]:
# guessed 631
# guessed 2156 (too low)

In [ ]:
print(valves)

In [179]:
#non_zero_valves[start] = 0

In [181]:
for k,v in paths.items():
    for tmp in v:
        if k not in paths[tmp]:
            print("Caves are not reflexive")


In [182]:
print(non_zero_valves)

{'RQ': 13, 'GU': 21, 'AZ': 12, 'WQ': 23, 'IE': 22, 'BT': 24, 'RU': 19, 'IF': 7, 'ME': 18, 'MH': 8, 'FI': 11, 'TD': 16, 'MU': 15, 'CQ': 9, 'UN': 20}


In [183]:
print(shorter_caves)

{'RQ': {'BL': 1, 'DJ': 1, 'BT': 2, 'NT': 2, 'KB': 3, 'GA': 3, 'UN': 3, 'NW': 4, 'DR': 4, 'XD': 4, 'RF': 4, 'MH': 5, 'CQ': 5, 'CN': 5, 'BA': 5, 'FS': 6, 'SS': 6, 'SY': 6, 'UU': 6, 'WQ': 6, 'GU': 6, 'HH': 7, 'MU': 7, 'VJ': 7, 'UM': 7, 'YP': 7, 'AA': 8, 'NA': 8, 'IE': 8, 'CO': 9, 'AS': 9, 'HK': 9, 'PQ': 9, 'PH': 9, 'XI': 9, 'XA': 9, 'FI': 10, 'AZ': 10, 'IF': 10, 'TD': 10, 'ME': 10, 'DI': 10, 'PI': 11, 'MR': 11, 'AR': 11, 'QF': 11, 'PE': 11, 'JL': 11, 'GC': 11, 'OQ': 11, 'WB': 11, 'BO': 11, 'AB': 12, 'RU': 13}, 'GU': {'VJ': 1, 'BA': 1, 'YP': 1, 'WQ': 2, 'RF': 2, 'IE': 2, 'UM': 3, 'CN': 3, 'UN': 3, 'XI': 3, 'XA': 3, 'XD': 4, 'NT': 4, 'IF': 4, 'DI': 4, 'DJ': 5, 'AR': 5, 'JL': 5, 'HK': 5, 'PE': 5, 'FI': 5, 'RQ': 6, 'WB': 6, 'AA': 6, 'AZ': 6, 'PI': 6, 'MR': 6, 'CO': 6, 'BL': 7, 'TD': 7, 'HH': 7, 'AS': 7, 'PQ': 7, 'QF': 7, 'GC': 7, 'BT': 8, 'OQ': 8, 'FS': 8, 'KB': 9, 'GA': 9, 'ME': 9, 'MH': 9, 'NW': 10, 'DR': 10, 'BO': 10, 'PH': 10, 'SS': 10, 'SY': 10, 'CQ': 11, 'AB': 11, 'NA': 11, 'MU': 11, 'U

In [ ]:
biggest = 0
def estimate_remaining_potential(rate,time_left,opened,valves):
    unopened = [v for k,v in valves.items() if k not in opened]
    unopened = sorted(unopened)[-(time_left)//2:]
    return (rate+sum(unopened))*time_left+100

def trySolution(loc,time_left,rate,released,opened):
    global biggest
    global shorter_caves
    global non_zero_valves
    global valves
    global paths
    
    # update
    released += rate
    time_left -= 1
    
    if time_left == 0:
        if released > biggest:
            biggest = released
            print(biggest)
        return released # may have to add current rate

    # be smart and leave if already lost
    if released + estimate_remaining_potential(rate,time_left,opened,valves) < biggest:
        return 0

    #print(released,time_left)
    extra = 0
    # random bullshit GO!
    if loc not in opened and valves[loc] > 0:
        #new_opened = copy.copy(opened)
        opened.add(loc)
        extra = trySolution(loc,time_left-1,rate+valves[loc],released,opened)
        opened.remove(loc)
    #if loc in shorter_caves:
    #    for k in non_zero_valves:
    #        v = shorter_caves[loc][k]
    #    #for k,v in shorter_caves[loc].items():
    #        if v <= time_left and v > 0:
    #            extra1 = trySolution(k,time_left-(v-1),rate,released+rate*(v-1),opened)
    #            extra = max(extra,extra1)
    if True:
        #print("we shouldn't be here")
        for new_locs in paths[loc]:
            extra1 = trySolution(new_locs,time_left,rate,released,opened)
            extra = max(extra,extra1)
    return extra


In [ ]:
biggest = 0
trySolution(start,30,0,0,set())


In [ ]:
def try2():
    #state = (loc,rate,released,time_left,flipped)
    state = {}
    